In [91]:
import pandas as pd
import numpy as np
from read_bayesnet import BayesNet

In [92]:
network = BayesNet('earthquake.bif')

In [93]:
network.probabilities['Burglary'].shape

(2, 2)

In [95]:
def marginalize(node,probs):
    probability = probs['prob']
    cpt=probs.drop([node,'prob'],axis=1)
    marginal = pd.DataFrame(columns=cpt.columns.tolist())
    probs = []
    length = cpt.shape[1]
    while cpt.shape[0] > 0:
        positions = [x for x in range(0,cpt.shape[0]) if sum(cpt.iloc[0]==cpt.iloc[x]) == cpt.shape[1]]

        probs.append(sum(probability[probability.index[positions]]))
        marginal = marginal.append(cpt[:1])

        cpt=cpt.drop(cpt.index[positions],axis=0)
        probability=probability.drop(probability.index[positions],axis=0)

    marginal.insert(length,'prob',probs)    
    return marginal

In [96]:
network.probabilities['Alarm']

,Alarm,Burglary,Earthquake,prob
0,True,True,True,0.950
1,False,True,True,0.050
2,True,False,True,0.290
3,False,False,True,0.710
4,True,True,False,0.940
5,False,True,False,0.060
6,True,False,False,0.001
7,False,False,False,0.999


In [97]:
m = marginalize('MaryCalls',network.probabilities['MaryCalls'])
m

,Alarm,prob
0,True,1.0
2,False,1.0


In [98]:
def product(probs1,probs2):
    intersec = list(filter(lambda x: x != 'prob' ,np.intersect1d(probs1.columns,probs2.columns)))
       
    probone = probs1['prob']
    probtwo = probs2['prob']

    cpt1 = probs1.drop('prob',axis=1)
    cpt2 = probs2.drop('prob',axis=1)
    
    prod = cpt1.join(cpt2.set_index(intersec),on=intersec)
    probs=[]
    length = prod.shape[1]
    
    for i in range(0,probone.shape[0]):
        for j in range(0,probtwo.shape[0]):
            if sum(cpt1[intersec].iloc[i] == cpt2[intersec].iloc[j]) == len(intersec):
                probs.append(probone.iloc[i] * probtwo.iloc[j])
    
    prod.insert(length,'prob',probs)
    return prod

In [99]:
p = product(network.probabilities['Alarm'],network.probabilities['MaryCalls'])
p

,Alarm,Burglary,Earthquake,MaryCalls,prob
0,True,True,True,True,0.66500
0,True,True,True,False,0.28500
1,False,True,True,True,0.00050
1,False,True,True,False,0.04950
2,True,False,True,True,0.20300
2,True,False,True,False,0.08700
3,False,False,True,True,0.00710
3,False,False,True,False,0.70290
4,True,True,False,True,0.65800
4,True,True,False,False,0.28200


In [100]:
def reduce(node,probs,evidence=None):
    if not evidence:
        return prob
    columns = probs.columns.tolist()
    try:
        i = columns.index(node)
        reduced = pd.DataFrame(list(filter(lambda x: x[i] == evidence,probs.values)),columns=columns)
        return reduced
    except:
        print("{} is not found in the probabilities".format(node))


In [101]:
network.probabilities['MaryCalls']

,MaryCalls,Alarm,prob
0,True,True,0.70
1,False,True,0.30
2,True,False,0.01
3,False,False,0.99


In [102]:
r = reduce('MaryCalls',network.probabilities['MaryCalls'],'True')
r

,MaryCalls,Alarm,prob
0,True,True,0.70
1,True,False,0.01


In [103]:
def observe(network,node,evidence):
    for n in network.nodes:
        if node in network.probabilities[n].columns.tolist():
            network.probabilities[n] = reduce(node,network.probabilities[n],evidence)

def marginalization(network,node):
    nodes = list(set(network.nodes).symmetric_difference(set([node])))
    for n in nodes:
        if node in network.probabilities[n].columns.tolist():
            network.probabilities[node] = product(network.probabilities[node],network.probabilities[n])

In [104]:
for marg_var in marg_vars:
        marg_net = []
        rm_nodes = []
        for i, node in enumerate(network):
            if marg_var in node.columns:
                marg_net.append(node)
                rm_nodes.append(i)

NameError: name 'marg_vars' is not defined

In [105]:
def query(observed,elim_order):
    
    for node in observed:
        observe(network,node,observed[node])
    
    for marg_var in elim_order:
        marg_net = []
        rm_nodes = []
        
        for i,node in enumerate(network.probabilities):
            if marg_var in network.probabilities[node]:
                marg_net.append(node)
                rm_nodes.append(node)
                
            if not len(marg_net) == 0:
                table = marg_net[0]
                
                rm_nodes.reverse()
                for i in rm_nodes:
                    del network.probabilities[node]
                
                for cpt in marg_net[1:]:
                    table = product(table, cpt)

                marginalized_table = marginalize(marg_var,network.probabilities[table])
                network.probabilities["factor{}".format(i)] = (marginalized_table)
                
                
    prod = [x for i,x in enumerate(network.probabilities) if i ==0]
    for node in network.probabilities[1:]:
        network.probabilities[prod] = product(network.probabilities[prod], node)
    return network.probabilities[prod]

In [109]:
observed = {'Burglary':'True'}
order = [x for x in network.nodes if x != 'MaryCalls']
order

['Burglary', 'Earthquake', 'Alarm', 'JohnCalls']

In [108]:
#query(observed,order)

In [90]:
order

['Burglary', 'Earthquake', 'Alarm', 'JohnCalls']